<h1><center> Data Cleaning File  <br><br> 
<font color='grey'> Cleaning Emissions and Traffic Data <br><br>


In [1]:
#setting libraries
import pandas as pd

In [2]:
#reading in emissions data files
emissions_16 = pd.read_excel("2016_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])
emissions_13 = pd.read_excel("2013_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])
emissions_19 = pd.read_excel("2019_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])

In [3]:
#create column names
new_col_names = ["city", "state", "zipcode", "county", "latitude", "longitude", "industry_sub", "industry_main", "total_emissions"]

In [4]:
#rename columns
for df in [emissions_16, emissions_13, emissions_19]:
    df.columns = new_col_names

In [5]:
#save list of relevant states and industry
rel_states = ["DC", "MD", "VA"]
rel_indust = "MN|NN|C|W"

In [6]:
#filter for dc, md, and va + relevant industries
emissions_16 = emissions_16.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')
emissions_13 = emissions_13.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')
emissions_19 = emissions_19.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')

In [7]:
#checking number of rows
[df.shape for df in [emissions_16, emissions_13, emissions_19]]

[(159, 9), (157, 9), (156, 9)]

In [8]:
#adding identifers before merging
emissions_13.loc[:, "time"] = 2013
emissions_16.loc[:, "time"] = 2016
emissions_19.loc[:, "time"] = 2019

C:\Users\helen\AppData\Local\Temp\ipykernel_10904\1078787289.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emissions_19.loc[:, "time"] = 2019


In [10]:
#merge
emissions = emissions_13.merge(emissions_16, how = "outer").merge(emissions_19, how = "outer")

In [19]:
#convert to wide format
emissions = emissions.pivot_table(
    index = list(emissions.columns[:-2]),
    columns = 'time',
    values = 'total_emissions'
).reset_index()

['city',
 'state',
 'zipcode',
 'county',
 'latitude',
 'longitude',
 'industry_sub',
 'industry_main']